In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from IPython.display import display
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import RobustScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score  
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import classification_report

### Область работы 2 (поиск  модели .... )

In [ ]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

path_train = 'train.csv' # содержит только имя файла, без имен папок !!!
wine = pd.read_csv(path_train)
wine

In [ ]:
display(wine.describe())
wine.info()

In [ ]:
wine[wine.duplicated()]

In [ ]:
wine['quality'] = [0 if i<7 else 1 for i in wine['quality']]

In [ ]:
wine.quality.value_counts()

In [ ]:
X = wine.drop(columns = 'quality')
Y = wine.quality
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,stratify = Y, random_state = 42)

In [ ]:
n_neighbors = np.arange(1, 30)
weights = ['uniform', 'distance']
scaling = [ MinMaxScaler(), StandardScaler(),RobustScaler(), Normalizer()]
preprocessing = [LogisticRegression(), KNeighborsClassifier(),GaussianNB()]
p = np.arange(1, 6)
C = np.logspace(-4, 4, 50)

In [ ]:
pipe = Pipeline([('preprocessing', MinMaxScaler()),
                   ('classifier', KNeighborsClassifier())])

param_grid = [{'preprocessing' : scaling,
               'classifier' : [KNeighborsClassifier()],
               'classifier__n_neighbors' : n_neighbors,
               'classifier__weights' : weights,
               'classifier__p' : p},
               {'preprocessing' : scaling,
                'classifier' : [GaussianNB()]},
               {'preprocessing': scaling,
                    'classifier': [LogisticRegression()],
                    'classifier__penalty': ['l2'],
                    'classifier__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                   'classifier__C' : C},
               {'preprocessing': scaling,
                'classifier': [LogisticRegression()],
                    'classifier__penalty': ['l1'],
                    'classifier__solver': ['liblinear', 'saga'],
                    'classifier__C' : C},
               {'preprocessing': scaling,
                'classifier': [LogisticRegression()],
                    'classifier__penalty': ['none'],
                    'classifier__solver': ['lbfgs','newton-cg', 'newton-cholesky', 'saga'],
                    'classifier__C' : C},
               {'preprocessing': scaling,
                'classifier': [LogisticRegression()],
                    'classifier__penalty' : ['elasticnet'],
                    'classifier__solver' : ['saga'],
                    'classifier__C' : C}]

grid_search = GridSearchCV(pipe, param_grid)
grid_search.fit(X_train, y_train)
y_pred_3 = grid_search.predict(X_test)

print("best params: \n{}\n".format(grid_search.best_params_))
print("accuarcy scor train: {:.6f}\n".format(grid_search.score(X_train, y_train))) 
print("accuarcy scor test: {:.6f}\n".format(grid_search.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score
print('recall:    ', roc_auc_score(y_test,  y_pred_3))
print('precision: ', precision_score(y_test,  y_pred_3))
print('f1:        ', f1_score(y_test,  y_pred_3))

In [ ]:
print(classification_report(y_test, y_pred_3))

In [ ]:
logistic_regression = LogisticRegression(class_weight='balanced')
pipe_2 = Pipeline([('preprocessing', StandardScaler()),
                    ('classifier', logistic_regression)])
param_grid_2 =[{'preprocessing': scaling,
                    'classifier__penalty': ['l2'],
                    'classifier__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                   'classifier__C' : C},
               {'preprocessing': scaling,
                    'classifier__penalty': ['l1'],
                    'classifier__solver': ['liblinear', 'saga'],
                    'classifier__C' : C},
               {'preprocessing': scaling,
                    'classifier__penalty': ['none'],
                    'classifier__solver': ['lbfgs','newton-cg', 'newton-cholesky', 'saga'],
                    'classifier__C' : C},
               {'preprocessing': scaling,
                    'classifier__penalty' : ['elasticnet'],
                    'classifier__solver' : ['saga'],
                    'classifier__C' : C}]
grid_search_2 = GridSearchCV(pipe_2, param_grid_2, return_train_score = True)
grid_search_2.fit(X_train, y_train)
y_pred_4 = grid_search_2.predict(X_test)
predicted_probabilities = grid_search_2.predict_proba(X_test)[:, 1]
threshold = 0.5
predicted_labels = (predicted_probabilities >= threshold).astype(int)
precision = precision_score(y_test, predicted_labels)
print("Метрика Precision с порогом", threshold, ":", precision)
print("Метрика recall", ":", recall_score(y_test, predicted_labels))

In [ ]:
grid_search_2.best_params_

In [ ]:
grid_result = pd.DataFrame(grid_search_2.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
pipe_2.fit(X_train,y_train)
y_pred_2 = pipe_2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_2))